In [4]:
from build.player.Player import Player
from build.Board import Result, Board
from build.Decorators import debug, log
from waiting import wait

import ipywidgets as widgets

In [6]:
class SmarterHuman(Player):
    """
    Human player class for human participation in a game
    """

    def __init__(self, representation_char):
        Player.__init__(self, representation_char)
        self.button_dict = dict()
        self.active_button = None

    # @debug
    # @log(humanReadable=True)
    def make_move(self, board: Board) -> int:
        """
        Prints a human readable game board starts input prompt
        :param board: current state of the board
        :return: list of selected column (first) and row (second)
        """

        if not self.button_dict:
            self.fill_button_dict(board)

        self.update_buttons(board)

        wait(lambda: self.is_button_clicked())

        return self.active_button

    def get_empty_button(self):
        button = widgets.Button(
            value=False,                                                        # TODO
            description=' ',
            tooltip='Click me for boolean change'
        )

        button.on_click(self.on_button_click)
        return button

    def update_buttons(self, board: Board) -> None:

        self.active_button = None

        for button_id, button in self.button_dict.items():

            placed_player = board.field[board.POSSIBLE_ACTIONS.get(button_id)]
            if placed_player is None:
                button.description = ' '
                button.disabled = False

            else:
                button.description = placed_player.representation_char
                button.disabled = True

    def is_button_clicked(self) -> bool:
        return self.active_button is not None

    def on_button_click(self, pressed_button: widgets.Button):

        pressed_button.description = self.representation_char
        self.active_button = pressed_button

        for button in self.button_dict.values():
            button.disabled = True

    # @debug
    def give_result(self, result: Result) -> None:
        """
        Prints human readable feedback
        :param result: Enum element for feedback
        """

        if Result.INVALID_MOVE == result:
            print("This position is blocked or not on the board!")

        elif Result.GAME_LOST == result:
            print("You have lost the game. :/")
            self.stats.incr_lost()

        elif Result.GAME_WON == result:
            print("You have won the game! :)")
            self.stats.incr_won()

        elif Result.GAME_DRAW == result:
            print("The game is over and no one won.")
            self.stats.incr_draw()

    def fill_button_dict(self, board: Board):

        columns, rows = board.SHAPE
        button_mash = []
        for column in range(columns):

            button_row = []
            for row in range(rows):

                btn = self.get_empty_button()
                button_row.append(btn)
                self.button_dict[row + row * column] = btn

            button_mash.append(widgets.VBox(button_row))

        display(widgets.HBox(button_mash))